In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Solicitar informações ao usuário
email = input("Digite seu e-mail: ")
senha = input("Digite sua senha: ")
filial = input('Qual filial: ')

# Inicializa a lista para armazenar dados dos pedidos
order_data = []
detalhes_produtos = []

def login(driver):
    print("Realizando login...")
    driver.get("https://one.bees.com/order-management/closed-orders")
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="signInName"]'))).send_keys(email)
    driver.find_element(By.XPATH, '//*[@id="next"]').click()
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="password"]'))).send_keys(senha)
    driver.find_element(By.XPATH, '//*[@id="next"]').click()
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="single-spa-application:@admin-portal/main-mfe"]/section/header/div[1]/div/div[1]/div[3]/div[2]/button'))).click()
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[@id="radix-3-trigger-REJECTED"]'))
    ).click()
    input()
def coletar_dados_pedidos(driver):
    """Coleta dados dos pedidos da página atual."""
    print("Coletando dados dos pedidos na página atual...")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pedidos = soup.findAll('tr', class_="c-fKvqNg c-fKvqNg-gsmDXe-rowClickable-true")

    for pedido in pedidos:
        colunas = pedido.findAll('td')
        texto_pedido = [coluna.get_text(strip=True) for coluna in colunas]
        numero_pedido = texto_pedido[0]  # Supondo que o número do pedido está na primeira coluna
        order_data.append(numero_pedido)

def navegar_paginas(driver):
    """Navega por todas as páginas de pedidos."""
    print("Navegando pelas páginas de pedidos...")
    for numero_pagina in range(1, 100):  # Ajuste o limite conforme necessário
        try:
            coletar_dados_pedidos(driver)

            # Espera até que o botão "próxima página" esteja clicável
            page_xpath = f'//li[@title="page {numero_pagina}"]'
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, page_xpath))
            )
            next_button.click()
            time.sleep(5)  # Aguarda o carregamento da página
        except Exception as e:
            print("Sem mais páginas.")
            break

def carregar_pedidos_existentes(caminho):
    """Carrega os números dos pedidos existentes da planilha."""
    try:
        df_existente = pd.read_csv(caminho)
        pedidos_existentes = set(df_existente["Número do pedido"].astype(str).str.strip().values)
        print("Pedidos existentes carregados da planilha.")
    except FileNotFoundError:
        pedidos_existentes = set()  # Se a planilha não existir, comece com um conjunto vazio
        print("Planilha não encontrada. Iniciando com conjunto vazio.")
        df_existente = pd.DataFrame()  # Cria um DataFrame vazio se não houver arquivo
    return pedidos_existentes, df_existente

def coletar_detalhes_pedido(driver, order, caminho):
    """Coleta detalhes de um pedido específico e salva no CSV."""
    print(f"Coletando detalhes do pedido: {order}")
    time.sleep(10)
    driver.get(f"https://one.bees.com/order-management/closed-orders/{order}")
    time.sleep(10)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]')))
    tbody = driver.find_element(By.XPATH, '//tbody[@role="rowgroup"]')
    produtos = tbody.find_elements(By.XPATH, './/tr[@role="row"]')
    detalhes_pedido = driver.find_element(By.CLASS_NAME, 'c-jUQOob')        

    total_elemento = driver.find_element(By.XPATH, '//div[@data-testid="order-total-value"]')
    total = total_elemento.find_element(By.TAG_NAME, 'div').text.strip()

    for produto in produtos:
        nome_sku = produto.find_element(By.XPATH, './/td[1]//p[1]').text.strip()
        sku = produto.find_element(By.XPATH, './/td[1]//p[2]').text.strip()
        quantidade_nao_entregue = produto.find_element(By.XPATH, './/td[2]').text.strip()
        preco = produto.find_element(By.XPATH, './/td[3]').text.strip()
        forma_pgto = detalhes_pedido.find_element(By.XPATH, '//p[@data-testid="method-0"]').text.strip()
        endereco_entrega = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[6]/div[3]/div[1]/div[5]/p[3]').text.strip()
        documento = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[6]/div[3]/div[1]/div[2]/p[2]').text.strip()
        motivo = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[6]/div[3]/div[2]/div[4]/p[2]').text.strip()
        data = detalhes_pedido.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[6]/div[3]/div[2]/div[1]/p[2]').text.strip()
        
        # Adiciona os detalhes do pedido
        detalhes_produtos.append({
            "Número do pedido": order,
            "Nome do produto": nome_sku,
            "SKU": sku,
            "Quantidade não entregue": quantidade_nao_entregue,
            "Preço": preco,
            "Forma de Pagamento": forma_pgto,
            "Endereço de Entrega": endereco_entrega,
            "Documento": documento,
            "Motivo": motivo,
            "Total do pedido": total,
            "Data": data
        })

        # Salva imediatamente os detalhes coletados em um DataFrame e no arquivo CSV
        df_novos = pd.DataFrame(detalhes_produtos)
        df_novos.to_csv(caminho, index=False, encoding='utf-8-sig')
        print(f"Detalhes do pedido {order} salvos com sucesso!")

def pedidos(driver, pedidos_existentes, df_existente, caminho):
    print("Iniciando a coleta de detalhes dos pedidos...")
    pedidos_novos = []  # Para armazenar pedidos que não estão na planilha
    for order in order_data:
        order = order.strip()  # Remove espaços em branco ao redor
        print(f"Verificando o pedido: {order}")  # Debug: Verifica cada pedido
        if order in pedidos_existentes:
            print(f"Pedido {order} já existe na planilha. Pulando...")
            continue  # Pula para o próximo pedido se já estiver na planilha
        else:
            pedidos_novos.append(order)  # Adiciona pedidos novos à lista

    print("Pedidos que não estão na planilha:", pedidos_novos)  # Imprime os pedidos novos

    for order in pedidos_novos:
        coletar_detalhes_pedido(driver, order, caminho)  # Coleta detalhes apenas se o pedido não está na planilha

# Inicializa o driver
driver = webdriver.Chrome()

# Carrega os pedidos existentes
caminho = f'Pedidos_cancelados_{filial}.csv'
pedidos_existentes, df_existente = carregar_pedidos_existentes(caminho)

# Realiza o login
login(driver)

# Navega pelas páginas de pedidos
navegar_paginas(driver)

# Coleta os dados dos pedidos
pedidos(driver, pedidos_existentes, df_existente, caminho)

# Fecha o navegador
driver.quit()


Pedidos existentes carregados da planilha.
Realizando login...
Navegando pelas páginas de pedidos...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Coletando dados dos pedidos na página atual...
Sem mais páginas.
Iniciando a coleta de detalhes dos pedidos...
Verificando o pedido: 9194499451
Verificando o pedido: 9193881546
Verificando o pedido: 9194072785
Verificando o pedido: 9193351439
Verificando o pedido: 9193334387
Verificando o pedido: 9193117467
Verificando o pedido: 9193190212
Verificando o pedido: 9192612236
Verificando o pedido: 9192764484
Verificando o pedido: 9192477255
Verificando o pedido: 9192480842
Verificando o pedido: 9192558762
Verificando o pedido: 9191954902
Verificando o pedido: 9191973907
Verificando o pedido: 9191862124
Verificando

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.205)
Stacktrace:
	GetHandleVerifier [0x00007FF7592AFB05+28789]
	(No symbol) [0x00007FF7592186E0]
	(No symbol) [0x00007FF7590B592A]
	(No symbol) [0x00007FF75908F505]
	(No symbol) [0x00007FF759136477]
	(No symbol) [0x00007FF75914EF42]
	(No symbol) [0x00007FF75912F1E3]
	(No symbol) [0x00007FF7590FA938]
	(No symbol) [0x00007FF7590FBAA1]
	GetHandleVerifier [0x00007FF7595E933D+3410093]
	GetHandleVerifier [0x00007FF7595FE7DD+3497293]
	GetHandleVerifier [0x00007FF7595F2A73+3448803]
	GetHandleVerifier [0x00007FF759377BBB+848171]
	(No symbol) [0x00007FF759223C3F]
	(No symbol) [0x00007FF75921F6E4]
	(No symbol) [0x00007FF75921F87D]
	(No symbol) [0x00007FF75920ED49]
	BaseThreadInitThunk [0x00007FFAE8CEE8D7+23]
	RtlUserThreadStart [0x00007FFAE9E9FBCC+44]
